In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline


In [ ]:
titanic_train = pd.read_csv('/content/drive/My Drive/data/dataset/titanic/train.csv')
titanic_test = pd.read_csv('/content/drive/My Drive/data/dataset/titanic/test.csv')


In [ ]:
def titanic_data(raw_data):
    tmp = []
    for each in raw_data['Sex']:
        if each == 'female':
            tmp.append(0)
        elif each == 'male':
            tmp.append(1)
        else:
            tmp.append(np.nan)
            

    tmp_emp =[]    
    for each in raw_data['Embarked']:
        if each == 'S':
            tmp_emp.append(0)
        elif each == 'C':
            tmp_emp.append(1)
        elif each == 'Q':
            tmp_emp.append(2)
        else: tmp_emp.append(0)

            
    raw_data['Embarked'] = tmp_emp
    raw_data['sex'] = tmp
    #raw_data['Age'] = tmp_age
    #raw_data['Survived'] = raw_data['Survived'].astype('float')
    raw_data['Pclass'] = raw_data['Pclass'].astype('float')
    raw_data['sex'] = raw_data['sex'].astype('float')
    raw_data['SibSp'] = raw_data['SibSp'].astype('float')
    raw_data['Parch'] = raw_data['Parch'].astype('float')
    raw_data['Fare'] = raw_data['Fare'].astype('float')
    raw_data['Age'] = raw_data['Age'].astype('float')
    raw_data['Name'] = raw_data['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
    
    name_age = pd.DataFrame(raw_data.groupby('Name').Age.median())
    raw_data['Age'].fillna(-1, inplace=True)

    for i in raw_data[raw_data['Age']== -1].index:
        for j in name_age.index:
            if raw_data.loc[i]['Name'] == j:
                raw_data.loc[i]['Age']=name_age.loc[j]['Age']
                
    tmp_age = []
    for each in raw_data['Age']:
        if each < 10:
            tmp_age.append(0)
        elif (10 <=each) & (each<20) :
            tmp_age.append(1)
        elif (20<=each) & (each<30 ):
            tmp_age.append(2)
        elif (30 <=each) & (each<40) :
            tmp_age.append(3)
        elif (40 <=each) & (each<50) :
            tmp_age.append(4)
        elif (50 <=each) & (each<60) :
            tmp_age.append(5)
        elif (60 <=each) & (each<70) :
            tmp_age.append(6)
        elif (70 <=each) & (each<=80) :
            tmp_age.append(7)
    raw_data['Age'] = tmp_age

    
    
    
    tmp_name =[]    
    for each in raw_data['Name']:
        if each == 'Mr':
            tmp_name.append(5)
        elif each == 'Miss':
            tmp_name.append(4)
        elif each == 'Mrs':
            tmp_name.append(3)
        elif each == 'Master':
            tmp_name.append(2)
        elif (each == 'Dr') or (each == 'Rev'):
            tmp_name.append(1)
        else: tmp_name.append(0)
            
    raw_data['Name'] = tmp_name
    # 미스터, 미세스와 같이 호칭별로 분류

    return raw_data

In [ ]:
titanic_train_data = titanic_data(titanic_train)
titanic_test_data = titanic_data(titanic_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(titanic_train_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare'  ,'Name']])
#X_scaled = scaler.transform(titanic_train_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare','Name']])
y_tot = titanic_train_data['Survived']

X_train, X_test, y_train, y_test = train_test_split(titanic_train_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare'  ,'Name']], 
                                                    titanic_train_data['Survived'].astype(float), 
                                                    stratify = titanic_train_data['Survived'].astype(float), random_state = 0,
                                                    train_size=0.6)

test_target = titanic_test_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare' ,'Name']]

In [ ]:
X_train.shape

(534, 7)

In [ ]:
import os 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'

checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose = 1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10)
#딥러닝을 구동하는데 필요한 케라스 함수 호출 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import tensorflow as tf

#실행 할때마다 같은 결과를 출력하기 위해 설정 
np.random.seed(3)
tf.random.set_seed(3)
model = Sequential()
#멤버함수 add()를 이용해 인공지능 계층 추가 - Relu
model.add(Dense(30, input_dim = 7, activation = 'relu'))


# 시그모이드 계층 추가 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=500, batch_size = 50, verbose = 0,
                    callbacks = [early_stopping_callback, checkpointer])


Epoch 00001: val_loss improved from inf to 0.61950, saving model to ./model/01-0.6195.hdf5

Epoch 00002: val_loss improved from 0.61950 to 0.59835, saving model to ./model/02-0.5984.hdf5

Epoch 00003: val_loss improved from 0.59835 to 0.59733, saving model to ./model/03-0.5973.hdf5

Epoch 00004: val_loss improved from 0.59733 to 0.59013, saving model to ./model/04-0.5901.hdf5

Epoch 00005: val_loss improved from 0.59013 to 0.58384, saving model to ./model/05-0.5838.hdf5

Epoch 00006: val_loss improved from 0.58384 to 0.58076, saving model to ./model/06-0.5808.hdf5

Epoch 00007: val_loss improved from 0.58076 to 0.57344, saving model to ./model/07-0.5734.hdf5

Epoch 00008: val_loss improved from 0.57344 to 0.56893, saving model to ./model/08-0.5689.hdf5

Epoch 00009: val_loss improved from 0.56893 to 0.56105, saving model to ./model/09-0.5611.hdf5

Epoch 00010: val_loss did not improve from 0.56105

Epoch 00011: val_loss improved from 0.56105 to 0.55256, saving model to ./model/11-0.55

In [ ]:
print('Train Accuracy : %.4f'%(model.evaluate(X_train,y_train)[1]))
print('test Accuracy : %.4f'%(model.evaluate(X_test, y_test)[1]))

17/17 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8184
Train Accuracy : 0.8184
12/12 [==============================] - 0s 2ms/step - loss: 0.4739 - accuracy: 0.8011
test Accuracy : 0.8011


In [ ]:
result = model.predict(test_target)
result = list(map(lambda x: 0 if x < 0.5 else 1 , result))
output = pd.DataFrame({'PassengerId' : titanic_test_data['PassengerId'],
                      'Survived' : result})
output['Survived'] = output['Survived'].astype(int)
output.to_csv('my_submission_SVC.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
